In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

/Users/zachary.garcia/Learning/zeus-groos-ai/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [3]:
def refresh_access_token():
    res = requests.post(
        "https://www.strava.com/oauth/token",
        data={
            "client_id": os.getenv("STRAVA_CLIENT_ID"),
            "client_secret": os.getenv("STRAVA_CLIENT_SECRET"),
            "refresh_token": os.getenv("STRAVA_REFRESH_TOKEN"),
            "grant_type": "refresh_token",
        }
    )
    print(res.status_code)
    print(res.json())
    res.raise_for_status()
    return res.json()["access_token"]

In [11]:
def get_strava_activities(access_token, per_page=5):
    url = "https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"per_page": per_page}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [14]:
token_data = refresh_access_token()

strava_data = get_strava_activities(token_data)


200
{'token_type': 'Bearer', 'access_token': '75fd1dc9f99b9743d0e95a1012c6f9510f0ece36', 'expires_at': 1748581631, 'expires_in': 18617, 'refresh_token': 'c280cb6536c00d9cbcd05b660423cb5ee91e34c8'}


In [22]:
type(strava_data[1])

dict

In [27]:
import pandas pd

df = pd.DataFrame(strava_data)


SyntaxError: invalid syntax (1239936607.py, line 1)